In [1]:
import xspec as x
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from astropy.io import fits



In [2]:

#Load relxill and test it

# --- STEP 1: DEFINE PATHS ---
# Path to the folder where you compiled relxill (contains lmodel.dat)
relxill_dir = "/home/kyle/software/relxill"

# Path to the folder containing the heavy FITS tables (often the same dir)
relxill_tables = "/home/kyle/software/relxill"

# --- STEP 2: SET ENVIRONMENT VARIABLE ---
# This tells relxill where to look for the tables so you don't have to copy them
os.environ["RELXILL_TABLE_PATH"] = relxill_tables

# --- STEP 3: LOAD THE MODEL ---
# This mimics the "lmod relxill /path" command
print("Loading Relxill...")
x.AllModels.lmod("relxill", relxill_dir)

# --- STEP 4: USE IT ---
# Now you can use 'relxill', 'relxillCp', 'relxilllp', etc.
# Example: simple relativistic reflection
m = x.Model("tbabs * relxillcp")

# Verify it loaded
print(m.componentNames)

Loading Relxill...
Model package relxill successfully loaded.
tbvabs Version 2.3
Cosmic absorption with grains and H2, modified from
Wilms, Allen, & McCray, 2000, ApJ 542, 914-924
Questions: Joern Wilms
joern.wilms@sternwarte.uni-erlangen.de
joern.wilms@fau.de

http://pulsar.sternwarte.uni-erlangen.de/wilms/research/tbabs/

PLEASE NOTICE:
To get the model described by the above paper
you will also have to set the abundances:
   abund wilm

Note that this routine ignores the current cross section setting
as it always HAS to use the Verner cross sections as a baseline.
['TBabs', 'relxillCp']
 *** loading RELXILL model (version 2.5) *** 

Model TBabs<1>*relxillCp<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   relxillCp  Incl       deg      30.0000      frozen
   3    2   relxillCp  a                   0.998000     +/-  0.0          
   4    2   relxillCp  Rin  

In [3]:
#load tbnew and test it

tbnew_dir = "/home/kyle/absmodel/"

x.AllModels.lmod("absmodel", tbnew_dir)

m = x.Model("tbnew*diskbb")



Model package absmodel successfully loaded.

Model tbnew<1>*diskbb<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   tbnew      nH         10^22    1.00000      +/-  0.0          
   2    1   tbnew      He                  1.00000      frozen
   3    1   tbnew      C                   1.00000      frozen
   4    1   tbnew      N                   1.00000      frozen
   5    1   tbnew      O                   1.00000      frozen
   6    1   tbnew      Ne                  1.00000      frozen
   7    1   tbnew      Na                  1.00000      frozen
   8    1   tbnew      Mg                  1.00000      frozen
   9    1   tbnew      Al                  1.00000      frozen
  10    1   tbnew      Si                  1.00000      frozen
  11    1   tbnew      S                   1.00000      frozen
  12    1   tbnew      Cl                  1.00000      frozen
  13    1   tbnew      Ar                  1.00000      frozen
  14    1   

tbvabs Version 2.3
Cosmic absorption with grains and H2, modified from
Wilms, Allen, & McCray, 2000, ApJ 542, 914-924
Questions: Joern Wilms
joern.wilms@sternwarte.uni-erlangen.de
joern.wilms@fau.de
http://pulsar.sternwarte.uni-erlangen.de/wilms/research/tbabs/

PLEASE NOTICE:
To get the model described by the above paper
you will also have to set the abundances:
   abund wilm

Note that this routine ignores the current cross section setting
as it always HAS to use the Verner cross sections as a baseline.


In [4]:
# --- 1. SETUP & DATA LOADING ---
# 1. Clear any existing data/models
x.AllData.clear()
x.AllModels.clear()

# 2. Define your file paths for clarity
# Base paths for full spectra
rgs1_spec = "../products/0865600201/rgs/filt/P0865600201R1S004SRSPEC1001.FIT"
rgs1_o2_spec = "../products/0865600201/rgs/filt/P0865600201R1S004SRSPEC2001.FIT"
rgs1_bkg = "../products/0865600201/rgs/filt/P0865600201R1S004BGSPEC1001.FIT"

rgs2_spec = "../products/0865600201/rgs/filt/P0865600201R2S005SRSPEC1001.FIT"
rgs2_o2_spec = "../products/0865600201/rgs/filt/P0865600201R2S005SRSPEC2001.FIT"
rgs2_bkg = "../products/0865600201/rgs/filt/P0865600201R2S005BGSPEC1001.FIT"

# Response matrices
rgs1_resp = "../products/0865600201/rgs/filt/P0865600201R1S004RSPMAT1001.FIT"
rgs1_o2_resp = "../products/0865600201/rgs/filt/P0865600201R1S004RSPMAT2001.FIT"

rgs2_resp = "../products/0865600201/rgs/filt/P0865600201R2S005RSPMAT1001.FIT"
rgs2_o2_resp = "../products/0865600201/rgs/filt/P0865600201R2S005RSPMAT2001.FIT"

# Time-interval datasets
time_intervals = {
    "Dipping": {
        "rgs1_spec": "../products/0865600201/rgs/time_intervals/Dipping/rgs1_src_o1_Dipping.fits",
        "rgs1_bkg": "../products/0865600201/rgs/time_intervals/Dipping/rgs1_bkg_o1_Dipping.fits",
        "rgs1_resp": "../products/0865600201/rgs/time_intervals/Dipping/rgs1_o1_Dipping.rmf",
        "rgs2_spec": "../products/0865600201/rgs/time_intervals/Dipping/rgs2_src_o1_Dipping.fits",
        "rgs2_bkg": "../products/0865600201/rgs/time_intervals/Dipping/rgs2_bkg_o1_Dipping.fits",
        "rgs2_resp": "../products/0865600201/rgs/time_intervals/Dipping/rgs2_o1_Dipping.rmf",
    },
    "Persistent": {
        "rgs1_spec": "../products/0865600201/rgs/time_intervals/Persistent/rgs1_src_o1_Persistent.fits",
        "rgs1_bkg": "../products/0865600201/rgs/time_intervals/Persistent/rgs1_bkg_o1_Persistent.fits",
        "rgs1_resp": "../products/0865600201/rgs/time_intervals/Persistent/rgs1_o1_Persistent.rmf",
        "rgs2_spec": "../products/0865600201/rgs/time_intervals/Persistent/rgs2_src_o1_Persistent.fits",
        "rgs2_bkg": "../products/0865600201/rgs/time_intervals/Persistent/rgs2_bkg_o1_Persistent.fits",
        "rgs2_resp": "../products/0865600201/rgs/time_intervals/Persistent/rgs2_o1_Persistent.rmf",
    },
}

In [5]:
# Choose which time interval to analyze
selected_interval = "Persistent"

interval_data = time_intervals[selected_interval]

x.AllData(f"1:1 {interval_data['rgs1_spec']} 2:2 {interval_data['rgs2_spec']}")

s1 = x.AllData(1)
s2 = x.AllData(2)

s1.background = interval_data['rgs1_bkg']
s2.background = interval_data['rgs2_bkg']

s1.response = interval_data['rgs1_resp']
s2.response = interval_data['rgs2_resp']


2 spectra  in use
 
Spectral Data File: ../products/0865600201/rgs/time_intervals/Persistent/rgs1_src_o1_Persistent.fits  Spectrum 1
Net count rate (cts/s) for Spectrum:1  1.303e-01 +/- 3.825e-03
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-2771
  Telescope: XMM Instrument: RGS1  Channel Type: PI
  Exposure Time: 1.918e+04 sec
 Using fit statistic: chi
 No response loaded.

Spectral Data File: ../products/0865600201/rgs/time_intervals/Persistent/rgs2_src_o1_Persistent.fits  Spectrum 2
Net count rate (cts/s) for Spectrum:2  1.670e-01 +/- 3.891e-03
 Assigned to Data Group 2 and Plot Group 2
  Noticed Channels:  1-2733
  Telescope: XMM Instrument: RGS2  Channel Type: PI
  Exposure Time: 1.922e+04 sec
 Using fit statistic: chi
 No response loaded.

***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
Net count rate (cts/s) for Spectrum:1  1.289e-01 +/- 3.825e-03 (99.0 % total)
***Warning!  One or more spectra are missing

In [6]:

# Load and display exposure times
print(f"\n{'='*60}")
print(f"EXPOSURE TIMES FOR: {selected_interval}")
print(f"{'='*60}")

s1_exp = s1.exposure
s2_exp = s2.exposure

print(f"RGS1 Exposure: {s1_exp:.2f} seconds ({s1_exp/1000:.2f} ks)")
print(f"RGS2 Exposure: {s2_exp:.2f} seconds ({s2_exp/1000:.2f} ks)")
print(f"Mean Exposure: {(s1_exp + s2_exp)/2/1000:.2f} ks")
print(f"{'='*60}\n")


EXPOSURE TIMES FOR: Persistent
RGS1 Exposure: 19178.22 seconds (19.18 ks)
RGS2 Exposure: 19217.01 seconds (19.22 ks)
Mean Exposure: 19.20 ks



In [7]:
# Set Global XSPEC Settings
x.Plot.device = "/null"
x.Plot.xAxis = "ang" # Or "angstrom" for RGS
x.AllData.ignore("**-3 .0 25.0-**")
x.Xset.xsect = "vern"
x.Xset.abund = "wilm"

# --- 2. MODEL DEFINITION ---
# Model: constant * tbfeo * (nthcomp + diskbb)
m1 = x.Model("constant * tbfeo * (nthcomp + diskbb + bbodyrad)")
m2 = x.AllModels(2)

# --- 3. APPLYING CORRECTED PARAMETERS FROM LATEST PN FIT ---

# 1. Instrument Normalization (constant)
m1(1).values = 1.0
m1(1).frozen = True
m2(1).values = 0.86781
m2(1).frozen = False

# 2. Absorption with variable O and Fe (tbfeo)
m1(2).values = 0.28802  # Best-fit nH from PN
m1(2).frozen = False     # Keep fixed to see O/Fe line residuals
m1(3).values = 0.1     # Oxygen abundance
m1(3).frozen = False
m1(4).values = 0.1      # Iron abundance
m1(4).frozen = False
m1(5).values = 0.0      # Redshift
m1(5).frozen = True

# 3. nthComp Continuum
m1(6).values = 1.49188  # Best-fit Gamma from PN
m1(6).frozen = True     # Keep shape fixed
m1(7).values = 1.62383  # Best-fit kT_e from PN
m1(7).frozen = True     # Keep shape fixed
m1(8).link   = "p12"    # kT_bb linked to diskbb Tin (p12)
m1(9).values = 1        # inp_type (diskbb)
m1(9).frozen = True
m1(10).values = 0.0     # Redshift
m1(10).frozen = True
m1(11).values = 0.00224 # Best-fit norm from PN
m1(11).frozen = True   # UNFROZEN to allow for flux scaling

# 4. diskbb Continuum
m1(12).values = 0.71659 # Best-fit Tin from PN
m1(12).frozen = True    # Keep shape fixed
m1(13).values = 0.89052 # Best-fit norm from PN
m1(13).frozen = True   #norm

# --- 4. FITTING ---
x.Fit.statMethod = "chi"
x.Fit.query = "yes"
x.Fit.perform()

     3 channels (1,3) ignored in spectrum #     1
     3 channels (1,3) ignored in spectrum #     2
      No channels ignored (no channels in specified range)
      No channels ignored (no channels in specified range)
  1204 channels (1568-2771) ignored in spectrum #     1
  1163 channels (1571-2733) ignored in spectrum #     2

 Cross Section Table set to vern:  Verner, Ferland, Korista, and Yakovlev 1996
 Solar Abundance Vector set to wilm:  Wilms, J., Allen, A. & McCray, R. ApJ 542 914 (2000) (abundances are set to zero for those elements not included in the paper).

Model constant<1>*TBfeo<2>(nthComp<3> + diskbb<4> + bbodyrad<5>) Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
                           Data group: 1
   1    1   constant   factor              1.00000      +/-  0.0          
   2    2   TBfeo      nH         10^22    1.00000      +/-  0.0          
   3    2   TBfeo      O                   1.00000      frozen
   4    2   TBfeo 

In [8]:
# # python
# # Set Global XSPEC Settings
# x.Plot.device = "/null"
# x.Plot.xAxis = "ang"              # wavelength units for RGS
# x.AllData.ignore("**-6.0 25.0-**")# ignore outside 6--25 Angstrom
# x.Xset.xsect = "vern"
# x.Xset.abund = "wilm"
#
# # --- MODEL DEFINITION: constant * tbfeo * diskbb ---
# m1 = x.Model("constant * tbfeo * diskbb")
# m2 = x.AllModels(2)  # handle for group 2
#
# # --- APPLY PARAMETERS (using PN-fit informed values) ---
#
# # 1. Instrument Normalization (constant)
# m1(1).values = 1.0
# m1(1).frozen = True
# m2(1).values = 0.86781
# m2(1).frozen = False
#
# # 2. Absorption with variable O and Fe (tbfeo)
# # Parameter ordering assumed: 2 = nH, 3 = O, 4 = Fe, 5 = redshift
# # m1(2).values = 0.28802   # nH (10^22 cm^-2)
# # m1(2).frozen = False
# # m1(3).values = 0.1       # O abundance
# # m1(3).frozen = False
# # m1(4).values = 0.1       # Fe abundance
# # m1(4).frozen = False
# # m1(5).values = 0.0       # redshift
# # m1(5).frozen = True
#
# # 3. diskbb Continuum
# # Parameter ordering assumed: 6 = Tin, 7 = norm
# # m1(6).values = 0.71659   # Tin (keV)
# # m1(6).frozen = True
# # m1(7).values = 0.89052   # norm
# # m1(7).frozen = True
#
# # --- FITTING ---
# x.Fit.statMethod = "chi"
# x.Fit.query = "yes"
# x.Fit.perform()

In [9]:

# --- NEATLY PRINT BEST FIT OUTPUT (Dual Data Group) ---
print("\n" + "="*95)
print(f"{'GROUP':<5} | {'C#':<3} | {'P#':<3} | {'COMPONENT':<12} | {'PARAMETER':<12} | {'VALUE':<12} | {'ERROR'}")
print("-" * 95)

# Track the global parameter index manually
for group_idx in [1, 2]:
    m = x.AllModels(group_idx)
    global_par_idx = 1
    
    for comp_idx, comp_name in enumerate(m.componentNames, start=1):
        comp = getattr(m, comp_name)
        for par_name in comp.parameterNames:
            par = getattr(comp, par_name)
            
            # Formatting Value and Error string
            val = par.values[0]
            if par.link != "":
                err_str = f"LINKED ({par.link})"
            elif par.frozen:
                err_str = "FROZEN"
            else:
                err_str = f"+/- {par.sigma:.5f}" if par.sigma > 0 else "N/A"
            
            # Print row logic: 
            # Show all of Group 1. 
            # For Group 2, only show things that aren't redundant links (like constant norm)
            if group_idx == 1 or par.link == "" or comp_name == "constant":
                print(f"{group_idx:<5} | {comp_idx:<3} | {global_par_idx:<3} | {comp_name:<12} | {par_name:<12} | {val:<12.5f} | {err_str}")
            
            global_par_idx += 1

print("-" * 95)
print(f"Final {x.Fit.statMethod.upper()}(): {x.Fit.statistic:.2f}")
print(f"Degrees of Freedom: {x.Fit.dof}")
print(f"Reduced Statistic: {x.Fit.statistic/x.Fit.dof:.3f}")
print("="*75 + "\n")


GROUP | C#  | P#  | COMPONENT    | PARAMETER    | VALUE        | ERROR
-----------------------------------------------------------------------------------------------
1     | 1   | 1   | constant     | factor       | 1.00000      | FROZEN
1     | 2   | 2   | TBfeo        | nH           | 0.31312      | +/- 0.08981
1     | 2   | 3   | TBfeo        | O            | 0.94033      | +/- 0.76289
1     | 2   | 4   | TBfeo        | Fe           | 3.30640      | +/- 1.41418
1     | 2   | 5   | TBfeo        | redshift     | 0.00000      | FROZEN
1     | 3   | 6   | nthComp      | Gamma        | 1.49188      | FROZEN
1     | 3   | 7   | nthComp      | kT_e         | 1.62383      | FROZEN
1     | 3   | 8   | nthComp      | kT_bb        | 0.71659      | LINKED (= p12)
1     | 3   | 9   | nthComp      | inp_type     | 1.00000      | FROZEN
1     | 3   | 10  | nthComp      | Redshift     | 0.00000      | FROZEN
1     | 3   | 11  | nthComp      | norm         | 0.00224      | FROZEN
1     | 4   | 12 

In [10]:

x.Plot.setRebin(3, 3)

x.Plot("data")


# Spectrum 1
energy_x   = x.Plot.x(1)      # X-axis values
energy_err = x.Plot.xErr(1)   # X-axis error bars
data_y     = x.Plot.y(1)      # Data counts/rate
data_err   = x.Plot.yErr(1)   # Data errors
model_y    = x.Plot.model(1)  # The folded model values

# Spectrum 2
energy_x_2   = x.Plot.x(2)
energy_err_2 = x.Plot.xErr(2)
data_y_2     = x.Plot.y(2)
data_err_2   = x.Plot.yErr(2)
model_y_2    = x.Plot.model(2)
y_units = x.Plot.labels()[1]
print(y_units)

x.Plot("delchi")
residuals = x.Plot.y(1)
residuals_2 = x.Plot.y(2)

x.Plot("background")
bkg_y1 = x.Plot.y(1)
bkg_y2 = x.Plot.y(2)



counts s$^{-1}$ Hz$^{-1}$


In [11]:
# python
%matplotlib notebook
plt.style.use('default')
    
# Use layout="constrained" for automatic tight fitting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True,
                                   gridspec_kw={'height_ratios': [3, 1], 'hspace': 0.0},
                                   dpi=100, layout="constrained")

    # --- TOP PANEL: SPECTRUM ---
ax1.set_axisbelow(False)

# Spectrum 1
ax1.errorbar(
    energy_x, data_y, xerr=energy_err, yerr=data_err,
    fmt='o', ms=1.5, elinewidth=0.8, capsize=0,
    ls='none', label='rgs 1 data', alpha=0.5
)

# Spectrum 2
ax1.errorbar(
    energy_x_2, data_y_2, xerr=energy_err_2, yerr=data_err_2,
    fmt='o', ms=1.5, elinewidth=0.6, capsize=0,
    color='green', alpha=0.4, ls='none', label='rgs 2 data'
)

# Draw models
ax1.step(energy_x, model_y, where='mid', color='C1', lw=1.2, zorder=20, label='rgs 1 Model')
ax1.step(energy_x_2, model_y_2, where='mid', color='purple', lw=1.2, zorder=20, label='rgs 2 Model')

# Plot Backgrounds
ax1.step(energy_x, bkg_y1, where='mid', color='C1', ls='--', lw=1.0, alpha=0.3, label='rgs 1 Background')
ax1.step(energy_x_2, bkg_y2, where='mid', color='purple', ls='--', lw=1.0, alpha=0.3, label='rgs 2 Background')

ax1.set_yscale("log")
ax1.set_ylabel(y_units)
ax1.set_title("RGS Spectrum and Model")
ax1.legend(loc="upper right", fontsize=8, frameon=True)

# --- BOTTOM PANEL: RESIDUALS ---
ax2.step(energy_x, residuals, where='mid', label='rgs 1', lw=1.0)
ax2.step(energy_x_2, residuals_2, where='mid', label='rgs 2', color='green', lw=1.0)
ax2.axhline(0, color='red', linestyle='--', lw=1.0)

ax2.set_xlabel(r"Wavelength ($\AA$)")
ax2.set_ylabel(r"Residuals ($\chi$)")
ax2.set_ylim(-5, 5) # Adjust based on your data

plt.show()

<IPython.core.display.Javascript object>